# Personal tweets sentiment analysis

## This paper presents user interaction with the developed model

interaction interface:
1: You will need to enter screenname in the variable "user_screen_name"<br>
2: Enter the required number of current tweets to view the variable "tweets_count"<br>
3: Run all jupyter cells<br>

In [ ]:
user_screen_name = "realDonaldTrump"
tweets_count = 5

In [ ]:
import os
import _thread
import time
os.mkdir('test')
LOG_DIR = './test'
get_ipython().system_raw('tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'.format(LOG_DIR))
get_ipython().system_raw('./ngrok http 6006 &')
def print_time( threadName, delay):
   count = 0
   while count < 2*60*60:
      time.sleep(delay)
      count += 1
try:
   _thread.start_new_thread( print_time, ("Thread-1", 60, ) )
except:
   print ("Error: unable to start thread")
url = ! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
print(url[0]+"/#projector")

In [ ]:
%run main.py
main_func(user_screen_name, tweets_count)